## Wings of butterflies

The authors of scientific article on butterfly wing shapes [ 1 ] have made the data on which their analysis is based publicly available [ 2 ]. In this project we will reconstruct those data so that they are ready to be analyzed. 

The data used by the article's authors is stored as an XLSX file, with records spread over 10 sheets. The aim is to import some of the data from the XLSX file into Python and output a CSV file. 


[1] Jones, R.T., Poul, Y.L., Whibley, A.C., Mérot, C., ffrench‐Constant, R.H. and Joron, M. (2013), WING SHAPE VARIATION ASSOCIATED WITH MIMICRY IN BUTTERFLIES. Evolution, 67: 2323-2334. <a href = "doi:10.1111/evo.12114"> doi:10.1111/evo.12114 </a>

[2] <a href = "https://datadryad.org/stash/dataset/doi:10.5061/dryad.1s15q"> https://datadryad.org/stash/dataset/doi:10.5061/dryad.1s15q </a> and "WING paper Data for repository.xlsx" on JupyterHub.

In [1]:
import csv
import re
import matplotlib.pyplot as plt
import xlrd

### Task 1

Implementing the function `load_data`.

This function takes an excel filepath as input and returns a dictionary which contains the data from the file. Every sheet and its content in the excel file corresponds to a key value pair in the dictionary. Rows will be encoded as lists of values and therefore a sheet becomes a list of lists.

In [2]:
def load_data(excel_path):
    """
    Load data from excel file
    
    Parameters
    ----------
    excel_path : str
        Path to wing paper excel file (.xlsx)
    
    Returns
    -------
    data : dictionary of list of lists
        Table data from wing paper excel file for all sheets
    """
    
    data = {}
    # we will use `xlrd` library to read the excel file
    wing_data = xlrd.open_workbook(excel_path)
    # iterate over sheets
    for sheet_name in wing_data.sheet_names():
        # get excel sheet
        xl_sheet = wing_data.sheet_by_name(sheet_name)
        # iterate for each row and column to get the cells
        for row_index in range(0, xl_sheet.nrows):
            row_object = []
            for col_index in range(0, xl_sheet.ncols):
                cell_value = xl_sheet.cell(row_index, col_index).value
                if isinstance(cell_value, str):
                    # check if the number is float but encoded as str
                    if cell_value.lstrip('-').replace('.', '', 1).isdigit():
                        cell_value = int(cell_value) if float(cell_value).is_integer() else float(cell_value)
                elif isinstance(cell_value, float):
                    cell_value = int(cell_value) if cell_value.is_integer() else cell_value
                row_object.append(cell_value)
            # escape empty and description rows
            if set(row_object) == {''}:
                break
            data.setdefault(sheet_name, []).append(row_object)
    
    return data

In [3]:
DATA = load_data("WING paper Data for repository.xlsx")

In [4]:
# tests to make sure we have the right data structure
assert isinstance(DATA, dict)
assert all(isinstance(k, str) and isinstance(v, list) for k, vs in DATA.items() for v in vs)

Note that there are columns in some of the sheets that are text format, therefore `xlrd` library importes those as string automatically even though most of them are float numbers. 

Also, knowing that `xlrd` library automatically converts integer numbers to float (with .0 decimal point), that needed to be handled manually by checking if float number is an integer (using `is_integer()` function).

### Task 2

Implementing the function `landmark_coordinates_table`.

The sheet "Landmark coordinates" encodes wing color pattern landmark coordinates in a special format. We need to convert the contents of this sheet into a plain table format. The column headings of this table will be `name, landmark 1 x, landmark 1 y, landmark 2 x, landmark 2 y, ...`. The enumeration of the landmarks corresponds to the position in the list of the special format. The `name` column will contain the name of the organism which the landmarks belong to.

In [5]:
def landmark_coordinates_table(data):
    """
    Transform landmark coordinates sheet data from special format to table format
    
    Parameters
    ----------
    data : dictionary of list of lists
        Data loaded from `load_data` function in Task 1
    
    Returns
    -------
    output : list of lists
        Transformed landmark coordinates from landmark coordinates sheet
    """
    
    output = []
    # creating the header
    landmark_data = data['Landmark coordinates']
    column_names = ['name']
    for i in range(1, 21):
        column_names.extend(["landmark %d x" % i, "landmark %d y" % i])
    # creating the body
    row_data = []
    row = []
    for item in landmark_data:
        if isinstance(item[0], str):
            if item[0] == "LM=20" or item[0].startswith("ID"):
                continue
            else:
                # getting name from IMAGE
                row.insert(0, int(item[0].split('.')[1].split("-")[0]))
                row_data.extend([row])
                row = []
        else:
            row.extend(item)
    
    output.append(column_names)
    output.extend(row_data)
    
    return output

In [6]:
DATA["Landmark coordinates"] = landmark_coordinates_table(DATA)

Note that landmark coordinates have the landmark names encoded into the IMAGE name, after the coordinates, therefore the name of the landmark was collected by splitting up the IMAGE name for each row.

### Task 3

Implementing the function `build_csv`.

Here we will output the data to a CSV file which contains the following data for each "name" in the "Experimental brood" sheet:

- name
- sex
- genotype
- phenotype
- wing perimeter
- landmark 1 x, landmark 1 y, landmark 2 x, ...

In [7]:
def build_csv(data, output_path):
    """
    Build and write a CSV file
    
    Parameters
    ----------
    data : dictionary of list of lists
        Data loaded from `load_data` function in Task 1 and modified in Task 2
    output_path : str
        Path to store CSV file to
    """
    
    # creating the header of the file
    column_names = ["name"] # we write name because in the "Experimental brood" sheet, "name" column has one additional space
    column_names.extend(data["Experimental brood"][0][1:])
    column_names.append("wing perimeter")
    column_names.extend(data["Landmark coordinates"][0][1:])
    # creating the body of the file
    row_data = []
    for items in data["Experimental brood"][1:]:
        row = []
        # add items from "Experimental brood" sheet to the row list
        row.extend(items)
        # goto "Wing perimeter" sheet to get the wing perimeter
        add_empty = True
        for wing_items in data["Wing perimeter"][1:]:
            if str(int(items[0])) in wing_items[0]:
                # add Perimeter to the row list
                row.append(wing_items[2])
                add_empty = False
                break
        # add empty cell to the row list
        if add_empty:
            row.append("")
        # goto "Landmark coordinates" to get the landmarks
        add_empty = True
        for landmark_items in data["Landmark coordinates"][1:]:
            if int(items[0]) == int(landmark_items[0]):
                # add landmarks to the row list
                row.extend(landmark_items[1:])
                add_empty = False
                break
        # add empty list to the row list
        if add_empty:
            row.extend([""] * 40) # 20 x + 20 y landmark coordinates = 40 empty cells
        row_data.extend([row])

    output = []
    output.append(column_names)
    output.extend(row_data)
    
    with open(output_path, "w") as file:
        writer = csv.writer(file, delimiter=',', lineterminator='\n')
        writer.writerows(output)

In [8]:
build_csv(DATA, "wings.csv")